In [1]:
from __future__ import division
import pickle
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import *
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_curve, auc
import pandas as pd
import scipy
from scipy import stats

In [2]:
X_train = pd.read_csv("data/X_train.csv")
Y_train = pd.read_csv("data/Y_train.csv")

In [4]:
X_train.columns

Index([u'Unnamed: 0', u'CustomerMD5Key', u'ReceivedDateTime', u'SCID',
       u'SelectedPackage', u'FirstDriverMaritalStatus', u'CarAnnualMileage',
       u'CarFuelId', u'CarUsageId', u'FirstDriverAge', u'CarInsuredValue',
       u'CarAge', u'FirstDriverDrivingLicenseNumberY', u'VoluntaryExcess',
       u'CarParkingTypeId', u'PolicyHolderNoClaimDiscountYears',
       u'FirstDriverDrivingLicenceType', u'CoverIsNoClaimDiscountSelected',
       u'CarDrivingEntitlement', u'CarTransmissionId', u'SocioDemographicId',
       u'PolicyHolderResidencyArea', u'AllDriversNbConvictions', u'TodayDate',
       u'RatedDriverNumber', u'IsPolicyholderAHomeowner', u'CarMakeId',
       u'DaysSinceCarPurchase', u'NameOfPolicyProduct', u'AffinityCodeId'],
      dtype='object')

In [8]:
X_train.shape, Y_train.shape

((2911318, 30), (2911318, 2))

In [9]:
del X_train['SelectedPackage']
del X_train['TodayDate']
del X_train['ReceivedDateTime']
del X_train['Unnamed: 0']
del Y_train['Unnamed: 0']
X_train['labels'] = Y_train['Converted']

In [10]:
X_train.shape

(2911318, 27)

In [11]:
X_train = X_train.dropna()

In [12]:
X_train.shape

(2911270, 27)

In [13]:
npp_unique = np.unique(X_train['NameOfPolicyProduct'])
updated_npp = np.array([np.arange(len(npp_unique))[np.array(val==npp_unique, dtype=bool)][0] for val in X_train['NameOfPolicyProduct']])
X_train['NameOfPolicyProduct'] = updated_npp

In [14]:
scid_unique = np.unique(X_train['SCID'])
updated_scid = np.array([np.arange(len(scid_unique))[np.array(val==scid_unique, dtype=bool) ][0] for val in X_train['SCID']])
X_train['SCID'] = updated_scid

In [15]:
ids = X_train['CustomerMD5Key']
#del X_train['CustomerMD5Key']

In [182]:
np.savez_compressed('processed_data_2.npz', X_train.ix[:, check.columns != 'labels'] , check['labels'])
np.savez_compressed('ids_list_2.npz', ids)

In [16]:
a = scipy.stats.itemfreq(ids)
a_ids = np.array([x[0] for x in a])
a_freq = np.array([int(x[1]) for x in a])

In [17]:
X_train.columns

Index([u'CustomerMD5Key', u'SCID', u'FirstDriverMaritalStatus',
       u'CarAnnualMileage', u'CarFuelId', u'CarUsageId', u'FirstDriverAge',
       u'CarInsuredValue', u'CarAge', u'FirstDriverDrivingLicenseNumberY',
       u'VoluntaryExcess', u'CarParkingTypeId',
       u'PolicyHolderNoClaimDiscountYears', u'FirstDriverDrivingLicenceType',
       u'CoverIsNoClaimDiscountSelected', u'CarDrivingEntitlement',
       u'CarTransmissionId', u'SocioDemographicId',
       u'PolicyHolderResidencyArea', u'AllDriversNbConvictions',
       u'RatedDriverNumber', u'IsPolicyholderAHomeowner', u'CarMakeId',
       u'DaysSinceCarPurchase', u'NameOfPolicyProduct', u'AffinityCodeId',
       u'labels'],
      dtype='object')

In [18]:
scipy.stats.itemfreq(X_train['NameOfPolicyProduct'])

array([[      0,       4],
       [      1,      70],
       [      2, 1596057],
       [      3,   28486],
       [      4,     169],
       [      5,     165],
       [      6,     167],
       [      7,  312613],
       [      8,    4947],
       [      9,   19904],
       [     10,  327289],
       [     11,  372205],
       [     12,   98402],
       [     13,   98108],
       [     14,   22580],
       [     15,     845],
       [     16,   29259]], dtype=int64)

In [22]:
keys = a_ids
values = a_freq
dictionary = dict(zip(keys, values))

In [23]:
ids_freq_matched = np.array([dictionary.get(x) for x in ids])

In [24]:
a_ids = np.array(a_ids)
ids = np.array(ids)

In [21]:
master = {i:[] for i in a_ids}

for idx, i in enumerate(ids):
    master[i].append(idx)

In [26]:
X_train['CustomerMD5Key'] = ids

In [27]:
X_train.columns

Index([u'CustomerMD5Key', u'SCID', u'FirstDriverMaritalStatus',
       u'CarAnnualMileage', u'CarFuelId', u'CarUsageId', u'FirstDriverAge',
       u'CarInsuredValue', u'CarAge', u'FirstDriverDrivingLicenseNumberY',
       u'VoluntaryExcess', u'CarParkingTypeId',
       u'PolicyHolderNoClaimDiscountYears', u'FirstDriverDrivingLicenceType',
       u'CoverIsNoClaimDiscountSelected', u'CarDrivingEntitlement',
       u'CarTransmissionId', u'SocioDemographicId',
       u'PolicyHolderResidencyArea', u'AllDriversNbConvictions',
       u'RatedDriverNumber', u'IsPolicyholderAHomeowner', u'CarMakeId',
       u'DaysSinceCarPurchase', u'NameOfPolicyProduct', u'AffinityCodeId',
       u'labels'],
      dtype='object')

In [39]:
customerkey_index = 0
pp_index = np.arange(len(X_train.columns))[np.array(X_train.columns) =='NameOfPolicyProduct' ][0]

In [51]:
master_pp = {i:[] for i in a_ids}

for idx, i in enumerate(np.array(X_train)):
    #if i[pp_index] not in master_pp[i[customerkey_index]]:
    master_pp[i[customerkey_index]].append(i[pp_index])

In [53]:
num_unique_pols = np.array([len(np.unique(master_pp.get(i))) for i in X_train['CustomerMD5Key']] )

In [67]:
num_total_pols =  np.array([len(master_pp.get(i)) for i in X_train['CustomerMD5Key']] )

In [ ]:
#num_unique_pols = np.array([ len(np.unique(np.array(X_train['NameOfPolicyProduct'])[master.get(t)]))  for t in ids])
#num_total_pols =  np.array([ len(X_train['NameOfPolicyProduct'][master.get(t)]) for t in ids])

In [ ]:
'CarAnnualMileage', 'CarFuelId', 'CarUsageId', 'CarParkingTypeId', 'CarMakeId', 'CarInsuredValue', 'CarAge'

In [88]:
customerkey_index = 0
# cam_index = np.arange(len(X_train.columns))[np.array(X_train.columns) =='CarAnnualMileage'][0]
# cmi_index = np.arange(len(X_train.columns))[np.array(X_train.columns) =='CarMakeId'][0]
ca_index = np.arange(len(X_train.columns))[np.array(X_train.columns) =='CarAge'][0]
civ_index = np.arange(len(X_train.columns))[np.array(X_train.columns) =='CarInsuredValue'][0]

# master_cam = {i:[] for i in a_ids}
# master_cmi = {i:[] for i in a_ids}
master_ca = {i:[] for i in a_ids}
master_civ = {i:[] for i in a_ids}

for idx, i in enumerate(np.array(X_train)):
    #master_cam[i[customerkey_index]].append(i[cam_index])
    #master_cmi[i[customerkey_index]].append(i[cmi_index])
    master_ca[i[customerkey_index]].append(i[ca_index])
    master_civ[i[customerkey_index]].append(i[civ_index])

In [89]:
#num_unique_cam = np.array([len(np.unique(master_cam.get(i))) for i in X_train['CustomerMD5Key']] )
#num_unique_cmi = np.array([len(np.unique(master_cmi.get(i))) for i in X_train['CustomerMD5Key']] )
num_unique_ca = np.array([len(np.unique(master_ca.get(i))) for i in X_train['CustomerMD5Key']] )
num_unique_civ = np.array([len(np.unique(master_civ.get(i))) for i in X_train['CustomerMD5Key']] )

In [92]:
scipy.stats.itemfreq(X_train['labels'])

array([[  0.00000000e+00,   2.88857000e+06],
       [  1.00000000e+00,   2.27000000e+04]])

In [115]:
car_con_1 = (num_unique_ca!=num_unique_civ)
car_con_2 = (num_unique_cam!=num_unique_civ)
car_con_3 = (num_unique_cmi!=num_unique_civ)
car_con_4 = (num_unique_cam!=num_unique_cmi)
car_con_5 = (num_unique_ca!=num_unique_cmi)
car_con_6 = (num_unique_cam!=num_unique_ca)

In [119]:
a_freq = scipy.stats.itemfreq(X_train['AffinityCodeId'])

array([[  0.00000000e+00,   1.13743400e+06],
       [  1.00000000e+00,   1.00000000e+00],
       [  3.00000000e+00,   5.17530000e+04],
       [  4.00000000e+00,   1.00000000e+00],
       [  6.00000000e+00,   1.00000000e+00],
       [  7.00000000e+00,   1.00000000e+00],
       [  9.00000000e+00,   2.00000000e+00],
       [  1.00000000e+01,   1.00000000e+00],
       [  1.10000000e+01,   1.21000000e+02],
       [  1.20000000e+01,   2.00000000e+00],
       [  1.30000000e+01,   5.36350000e+04],
       [  1.40000000e+01,   5.42430000e+04],
       [  1.70000000e+01,   1.00000000e+00],
       [  1.80000000e+01,   2.00000000e+00],
       [  1.90000000e+01,   1.00000000e+00],
       [  2.00000000e+01,   5.08550000e+04],
       [  2.20000000e+01,   5.41700000e+04],
       [  2.30000000e+01,   1.74280000e+04],
       [  2.60000000e+01,   5.20270000e+04],
       [  2.70000000e+01,   1.00000000e+00],
       [  2.80000000e+01,   9.94600000e+03],
       [  2.90000000e+01,   9.07410000e+04],
       [  

In [117]:
scipy.stats.itemfreq(np.array(X_train['NameOfPolicyProduct'])[np.array(X_train['labels']==1)] )

array([[    0,     4],
       [    1,    70],
       [    2, 14778],
       [    3,    58],
       [    7,  1037],
       [    8,  2855],
       [    9,   157],
       [   10,   797],
       [   11,  1168],
       [   12,   767],
       [   13,   212],
       [   15,   675],
       [   16,   122]], dtype=int64)

In [118]:
scipy.stats.itemfreq(np.array(X_train['NameOfPolicyProduct']))

array([[      0,       4],
       [      1,      70],
       [      2, 1596057],
       [      3,   28486],
       [      4,     169],
       [      5,     165],
       [      6,     167],
       [      7,  312613],
       [      8,    4947],
       [      9,   19904],
       [     10,  327289],
       [     11,  372205],
       [     12,   98402],
       [     13,   98108],
       [     14,   22580],
       [     15,     845],
       [     16,   29259]], dtype=int64)

In [101]:
X_train.columns

Index([u'CustomerMD5Key', u'SCID', u'FirstDriverMaritalStatus',
       u'CarAnnualMileage', u'CarFuelId', u'CarUsageId', u'FirstDriverAge',
       u'CarInsuredValue', u'CarAge', u'FirstDriverDrivingLicenseNumberY',
       u'VoluntaryExcess', u'CarParkingTypeId',
       u'PolicyHolderNoClaimDiscountYears', u'FirstDriverDrivingLicenceType',
       u'CoverIsNoClaimDiscountSelected', u'CarDrivingEntitlement',
       u'CarTransmissionId', u'SocioDemographicId',
       u'PolicyHolderResidencyArea', u'AllDriversNbConvictions',
       u'RatedDriverNumber', u'IsPolicyholderAHomeowner', u'CarMakeId',
       u'DaysSinceCarPurchase', u'NameOfPolicyProduct', u'AffinityCodeId',
       u'labels'],
      dtype='object')

In [ ]:
X, y = check.ix[:, check.columns != 'labels'] , check['labels']
X = StandardScaler().fit_transform(X)
print "done preprocessing"

clf = AdaBoostClassifier()
#score = sklearn.cross_validation.cross_val_score(clf, X, y, cv=10, n_jobs=5)
#print np.mean(score)
X_train_1, X_test_1, y_train_1, y_test_1  = train_test_split(X, y, test_size=.20)
clf.fit(X_train_1,y_train_1)
predictions = clf.predict(X_test_1)
sklearn.metrics.confusion_matrix(y_test_1, predictions)